In [ ]:
import uproot

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

import sys

# import plothist
# plt.style.use("belle2")
# from plothist import make_2d_hist, plot_2d_hist_with_projections

In [ ]:
plt.rcParams.update({
    "axes.labelsize": 16,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 14,
    "figure.titlesize": 20
})

In [ ]:
sys.path.append("/home/belle2/amubarak/Ds2D0enue_Analysis/07-Python_Functions/")

# Prep-Work

### Import Data

In [ ]:
import os
import pandas as pd
import uproot
from tqdm import tqdm

# === Load only selected branches ===
with open("/home/belle2/amubarak/Ds2D0enue_Analysis/03-Grid/Save_var.txt") as f:
    variables_to_load = [
        line.strip().strip(",").strip('"').strip("'")
        for line in f
        if line.strip() and not line.strip().startswith("#")
    ]

samples = ["Signal", "BB", "ccbar", "ddbar", "ssbar", "taupair", "uubar"]
GenEvents = ["Signal","BB","ccbar","ddbar","ssbar","taupair","uubar"]
Date = "0530"
Attempt = "0"

DataFrames = {}

# === Load each sample one by one with progress bar ===
for name in tqdm(samples, desc="Loading samples"):
    if name == "Signal":
        path = "/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:Dstree"
    else:
        path = f"/group/belle/users/amubarak/02-Grid/Sample_Grid/Ds2D0e-Generic_Ds_{Date}25_{Attempt}_{name}.root:Dstree"

    try:
        df = uproot.concatenate(path, filter_name=variables_to_load, library='pd')
        print(f"✔️ Loaded {name} with {len(df):,} entries")
        DataFrames[name] = df
    except Exception as e:
        print(f"❌ Failed to load {name}: {e}")
        DataFrames[name] = pd.DataFrame()

# === Merge background categories ===
background_samples = ["BB", "ccbar", "ddbar", "ssbar", "taupair", "uubar"]
DataFrames["All"] = pd.concat([DataFrames[s] for s in background_samples], ignore_index=True)

# === Combine uds light-quark backgrounds ===
DataFrames["uds"] = pd.concat(
    [DataFrames["uubar"], DataFrames["ddbar"], DataFrames["ssbar"]],
    ignore_index=True
)

In [ ]:
# # In this notebook we only process the main signal and the generic events,
# # for illustration purposes.
# # You can add other backgrounds after if you wish.
# samples = ["Signal","All","BB","ccbar","ddbar","ssbar","taupair","uubar","uds"]

# DataFrames = {}  # define empty dictionary to hold dataframes

# # Signal:
# DataFrames[samples[0]] =  uproot.concatenate("/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:Dstree",library='pd')
# # Background
# for s in samples[1:]: # loop over samples
#     DataFrames[s] =  uproot.concatenate("/group/belle2/users2022/amubarak/TopoAna/Completed_TopoAna/TopoAna_"+ s +".root:Dstree",library='pd')

In [ ]:
pd.set_option('display.max_rows', 200000)
pd.set_option('display.max_columns', 200000)

The line below is to look at the available variables.

In [ ]:
print(DataFrames.keys())

In [ ]:
DataFrames["ccbar"].columns.tolist()

### Setup
The code below will be used to apply cuts to the data.  
The range of the plots.

In [ ]:
# Electron ID
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['e_electronID']>=0.95]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['e_electronID']>=0.95]
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_em_electronID']>=0.95]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_gammaveto_em_electronID']>=0.95]

# Photon Conversion
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=0.1]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_gammaveto_M_Correction']>=0.1]

# Peaking Background Removal
#----------------------------
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_diff_D0pi']>=0.15)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=0.15)]

# # Vertex Fitting
# #----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb']>=0.01]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_chiProb']>=0.01]

# Dalitz Removal
#----------------------------
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["ccbar"]['Ds_pi0veto_M_Correction']>=0.16)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["Signal"]['Ds_pi0veto_M_Correction']>=0.16)]

# # Vertex Fit
# #----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames[samples[0]]['Ds_chiProb_Ds_rank']==1]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames[samples[1]]['Ds_chiProb_Ds_rank']==1]

# # D0 BDT
# #------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_extraInfo_FastBDT']>=0.58]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_extraInfo_FastBDT']>=0.58]

# Truth

In [ ]:
print(abs(DataFrames["Signal"][(DataFrames["Signal"]["Ds_mcPDG"].isna())])[['e_mcPDG','D0_mcPDG']].value_counts(normalize=True,dropna=False))
print(abs(DataFrames["All"][(DataFrames["All"]["Ds_mcPDG"].isna())])[['e_mcPDG','D0_mcPDG']].value_counts(normalize=True,dropna=False))

# Single Variable  
The code below is to look at any specific variable closely.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# === Settings ===
Bins = 50
Density = True
labels = [
    r'$isSignal(D_s^{+})=1$',
    r'$isSignal(D_s^{+})=0$',
    r'$NaN$'
]
colors = ["#4C6EB1", '#fd7f6f', 'purple']

df_signal = DataFrames["Signal"]

# === Variables and units ===
variables = {
    'e_chi':                            (r'$\chi \; (e)$', ''),
    'e_ndf':                            (r'$\mathrm{ndf} \; (e)$', ''),
    'e_omega':                          (r'$\omega \; (e)$', r'$[1/\mathrm{cm}]$'),
    'e_cos_theta':                      (r'$\cos\theta \; (e)$', ''),
    'e_phi':                            (r'$\phi \; (e)$', r'$[\mathrm{rad}]$'),
    'e_pt':                             (r'$p_{t} \; (e)$', r'$[\mathrm{GeV}/c]$'),
    'e_dr':                             (r'$dr \; (e)$', r'$[\mathrm{cm}]$'),
    'e_dz':                             (r'$dz \; (e)$', r'$[\mathrm{cm}]$'),
    'e_firstCDCLayer':                  (r'First CDC Layer (e)', ''),
    'e_firstPXDLayer':                  (r'First PXD Layer (e)', ''),
    'e_nPXDHits':                       (r'# PXD hits (e)', ''),
    'e_nCDCHits':                       (r'# CDC hits (e)', ''),
    'Ds_diff_D0K':                      (r'$\Delta m_{K}(D_s^+ - D^0)$', r'$[\mathrm{GeV}/c^2]$'),
    'Ds_chiProb':                       (r'$\chi^2_{\mathrm{prob}}(D_s^+)$', ''),
    'Ds_decayAngle_0':                  (r'$\cos\theta^*_{D^0}$', ''),
    'Ds_vertexDistanceOfDaughter_0':    (r'Vertex Distance (Daughter 0)', r'$[\mathrm{cm}]$'),
    'Ds_vertexDistanceOfDaughterErr_0': (r'Vertex Distance Err (Daughter 0)', r'$[\mathrm{cm}]$'),
    'Ds_daughterMotherDiffOf_0_p':      (r'$\Delta p(\mathrm{daughter}, D_s^+)$', r'$[\mathrm{GeV}/c]$'),
    'Ds_Dstarplus_M':                   (r'$m(D^{*+})$', r'$[\mathrm{GeV}/c^2]$'),
    'Ds_mcM_D0e_emass':                 (r'$m_{True,e}(D^0e^+)$', r'$[\mathrm{GeV}/c^2]$'),
    'Ds_mcM_D0e_pimass':                (r'$m_{True,\pi}(D^0e^+)$', r'$[\mathrm{GeV}/c^2]$'),
    'Ds_MminusMtrue_D0e_emass':         (r'$m - m_{True,e}(D^0e^+)$', r'$[\mathrm{GeV}/c^2]$'),
    'Ds_MminusMtrue_D0e_pimass':        (r'$m - m_{True,\pi}(D^0e^+)$', r'$[\mathrm{GeV}/c^2]$'),
}

# === Optional manual range override per variable ===
manual_ranges = {
    'e_chi': [0, 100],
    'e_ndf': [0, 50],
    'e_omega': [-0.2, 0.2],
    'e_cos_theta': [-1, 1],
    'e_phi': [-np.pi, np.pi],
    'e_pt': [0.0, 0.6],
    'e_dr': [0, 1],
    'e_dz': [-2, 2],
    'e_firstCDCLayer': [-1, 60],
    'e_firstPXDLayer': [-1, 6],
    'e_nPXDHits': [0, 10],
    'e_nCDCHits': [0, 100],
    'Ds_diff_D0K': [0.4, 2],
    'Ds_chiProb': [0.0, 1.0],
    'Ds_decayAngle_0': [0.0, np.pi],
    'Ds_vertexDistanceOfDaughter_0': [0.0, 5.0],
    'Ds_vertexDistanceOfDaughterErr_0': [0.0, 0.6],
    'Ds_daughterMotherDiffOf_0_p': [-0.25, 1.0],
    'Ds_Dstarplus_M': [1.8, 2.5],
    'Ds_mcM_D0e_emass': [1.8, 2.5],
    'Ds_mcM_D0e_pimass': [1.9, 2.5],
    'Ds_MminusMtrue_D0e_emass': [-0.1, 0.1],
    'Ds_MminusMtrue_D0e_pimass': [-0.1, 0.1],
}

for var, (label_x, unit) in variables.items():
    if var not in df_signal.columns:
        continue

    # Extract valid groups
    g1 = df_signal[df_signal['Ds_isSignal'] == 1][var].dropna()
    g2 = df_signal[df_signal['Ds_isSignal'] == 0][var].dropna()
    g3 = df_signal[df_signal['Ds_isSignal'].isna()][var].dropna()

    if any(len(g) < 10 for g in [g1, g2, g3]):
        continue

    # Range setup
    if var in manual_ranges:
        Range = manual_ranges[var]
    else:
        combined = pd.concat([g1, g2, g3])
        Range = list(np.percentile(combined, [1, 99]))

    # Bin width
    perBin = ((Range[1] - Range[0]) / Bins)

    # Plot
    plt.figure()
    plt.hist([g3, g2, g1],
             bins=Bins, range=Range, density=Density,
             histtype='step', linewidth=2,
             color=colors[::-1], label=labels[::-1])

    plt.xlabel(f"{label_x} {unit}")
    plt.ylabel(r'$Entries/(\;{:.3f}\;)$'.format(perBin))
    plt.title(var, loc="left")
    plt.title(r"$\bf Signal\;Events$", loc="right")
    plt.legend()
    plt.show()

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# === Settings ===
Bins = 50
Density = True
label_signal = r'Truth-matching Failed (Signal MC)'
label_generic = r'Truth-matching Failed (Generic MC)'
colors = ["#4C6EB1", '#fd7f6f']  # Signal, Generic

df_signal = DataFrames["Signal"][DataFrames["Signal"]["Ds_mcPDG"].isna()]
df_generic = DataFrames["All"][DataFrames["All"]["Ds_mcPDG"].isna()]

# === Variables and units ===
variables = {
    'e_chi':                            (r'$\chi \; (e)$', ''),
    'e_ndf':                            (r'$\mathrm{ndf} \; (e)$', ''),
    'e_omega':                          (r'$\omega \; (e)$', r'$[1/\mathrm{cm}]$'),
    'e_cos_theta':                      (r'$\cos\theta \; (e)$', ''),
    'e_phi':                            (r'$\phi \; (e)$', r'$[\mathrm{rad}]$'),
    'e_pt':                             (r'$p_{t} \; (e)$', r'$[\mathrm{GeV}/c]$'),
    'e_dr':                             (r'$dr \; (e)$', r'$[\mathrm{cm}]$'),
    'e_dz':                             (r'$dz \; (e)$', r'$[\mathrm{cm}]$'),
    'e_firstCDCLayer':                  (r'First CDC Layer (e)', ''),
    'e_firstPXDLayer':                  (r'First PXD Layer (e)', ''),
    'e_nPXDHits':                       (r'# PXD hits (e)', ''),
    'e_nCDCHits':                       (r'# CDC hits (e)', ''),
    'Ds_diff_D0K':                      (r'$\Delta m_{K}(D_s^+ - D^0)$', r'$[\mathrm{GeV}/c^2]$'),
    'Ds_chiProb':                       (r'$\chi^2_{\mathrm{prob}}(D_s^+)$', ''),
    'Ds_decayAngle_0':                  (r'$\cos\theta^*_{D^0}$', ''),
    'Ds_vertexDistanceOfDaughter_0':    (r'Vertex Distance (Daughter 0)', r'$[\mathrm{cm}]$'),
    'Ds_vertexDistanceOfDaughterErr_0': (r'Vertex Distance Err (Daughter 0)', r'$[\mathrm{cm}]$'),
    'Ds_daughterMotherDiffOf_0_p':      (r'$\Delta p(\mathrm{daughter}, D_s^+)$', r'$[\mathrm{GeV}/c]$'),
    'Ds_Dstarplus_M':                   (r'$m(D^{*+})$', r'$[\mathrm{GeV}/c^2]$'),
    'Ds_mcM_D0e_emass':                 (r'$m_{True,e}(D^0e^+)$', r'$[\mathrm{GeV}/c^2]$'),
    'Ds_mcM_D0e_pimass':                (r'$m_{True,\pi}(D^0e^+)$', r'$[\mathrm{GeV}/c^2]$'),
    'Ds_MminusMtrue_D0e_emass':         (r'$m - m_{True,e}(D^0e^+)$', r'$[\mathrm{GeV}/c^2]$'),
    'Ds_MminusMtrue_D0e_pimass':        (r'$m - m_{True,\pi}(D^0e^+)$', r'$[\mathrm{GeV}/c^2]$'),
}

# === Manual range override ===
manual_ranges = {
    'e_chi': [0, 100],
    'e_ndf': [0, 50],
    'e_omega': [-0.2, 0.2],
    'e_cos_theta': [-1, 1],
    'e_phi': [-np.pi, np.pi],
    'e_pt': [0.0, 0.6],
    'e_dr': [0, 1],
    'e_dz': [-2, 2],
    'e_firstCDCLayer': [-1, 60],
    'e_firstPXDLayer': [-1, 6],
    'e_nPXDHits': [0, 10],
    'e_nCDCHits': [0, 100],
    'Ds_diff_D0K': [0.4, 2],
    'Ds_chiProb': [0.0, 1.0],
    'Ds_decayAngle_0': [0.0, np.pi],
    'Ds_vertexDistanceOfDaughter_0': [0.0, 5.0],
    'Ds_vertexDistanceOfDaughterErr_0': [0.0, 0.6],
    'Ds_daughterMotherDiffOf_0_p': [-0.25, 1.0],
    'Ds_Dstarplus_M': [1.8, 2.5],
    'Ds_mcM_D0e_emass': [1.8, 2.5],
    'Ds_mcM_D0e_pimass': [1.9, 2.5],
    'Ds_MminusMtrue_D0e_emass': [-0.1, 0.1],
    'Ds_MminusMtrue_D0e_pimass': [-0.1, 0.1],
}

for var, (label_x, unit) in variables.items():
    if var not in df_signal.columns or var not in df_generic.columns:
        continue

    sig_vals = df_signal[var].dropna()
    gen_vals = df_generic[var].dropna()

    if len(sig_vals) < 10 or len(gen_vals) < 10:
        continue

    Range = manual_ranges[var]
    perBin = ((Range[1] - Range[0]) / Bins)

    plt.figure()
    plt.hist([sig_vals, gen_vals],
             bins=Bins, range=Range, density=Density,
             histtype='step', linewidth=2,
             color=colors, label=[label_signal, label_generic])

    plt.xlabel(f"{label_x} {unit}")
    plt.ylabel(r'$Entries/(\;{:.3f}\;)$'.format(perBin))
    plt.title(var, loc="left")
    plt.title(r"$\bf Truth-Matching\;Failure$", loc="right")
    plt.legend()
    plt.show()

In [ ]:
# === Settings ===
Stacked = True
Density = False
Bins = 50
Range = [0.0, 0.25]
perBin = ((Range[1] - Range[0]) / Bins) * 1000
print("Width Per Bin: {:.2f} MeV".format(perBin))

# Cut range on 'Ds_diff_D0pi'
cut_low = 0.14543 - (3 * 0.00041124)
cut_high = 0.14543 + (3 * 0.00041124)

# Variable to plot after cut
var = 'Ds_massDifference_0'

# Labels and colors
labels = [
    r'$c \bar{c}$',
    r'$u \bar{u}, \; d \bar{d}, \;s \bar{s}$',
    r'$BB$',
    r'$\tau^{+} \tau^{-}$'
]

# Truth-matching failure applied to each sample
data = [
    DataFrames["ccbar"].query("Ds_mcPDG.isna() and (Ds_diff_D0pi <= @cut_low or Ds_diff_D0pi >= @cut_high)")[var],
    DataFrames["uds"].query("Ds_mcPDG.isna() and (Ds_diff_D0pi <= @cut_low or Ds_diff_D0pi >= @cut_high)")[var],
    DataFrames["BB"].query("Ds_mcPDG.isna() and (Ds_diff_D0pi <= @cut_low or Ds_diff_D0pi >= @cut_high)")[var],
    DataFrames["taupair"].query("Ds_mcPDG.isna() and (Ds_diff_D0pi <= @cut_low or Ds_diff_D0pi >= @cut_high)")[var],
]

# === Plot ===
plt.hist(data[::-1],
         label=labels[::-1],
         density=Density,
         stacked=Stacked,
         bins=Bins,
         range=Range,
         histtype='step',
         linewidth=2)

# Titles
plt.title(r'$D_s^{+} \rightarrow [D^{0} \rightarrow K^{-} \pi^{+}] e^{+} \nu_{e}$' + '\n' + r'$\Delta m_{\pi}(D_s^{+} - D^{0}) \notin [0.142,\; 0.15] \; \mathrm{GeV}/c^{2}$', loc="left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc="right")

# Labels
plt.xlabel(r'$\Delta m_{e}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
plt.ylabel(r'$Entries/(\;{:.2f}\;MeV/c^2)$'.format(perBin))
plt.legend()
plt.show()


In [ ]:
# === Settings ===
Stacked = False
Density = True
Bins = 50
Range = [0.0, 0.25]
perBin = ((Range[1] - Range[0]) / Bins) * 1000
print("Width Per Bin: {:.2f} MeV".format(perBin))

# Cut range on 'Ds_diff_D0pi'
cut_low = 0.14543 - (3 * 0.00041124)
cut_high = 0.14543 + (3 * 0.00041124)

# Variable to plot after cut
var = 'Ds_massDifference_0'

# Labels and colors
labels = [
    r'$c \bar{c}$',
    r'$u \bar{u}, \; d \bar{d}, \;s \bar{s}$',
]

# Truth-matching failure applied to each sample
data = [
    DataFrames["ccbar"].query("Ds_mcPDG.isna() and (Ds_diff_D0pi <= @cut_low or Ds_diff_D0pi >= @cut_high)")[var],
    DataFrames["uds"].query("Ds_mcPDG.isna() and (Ds_diff_D0pi <= @cut_low or Ds_diff_D0pi >= @cut_high)")[var],
]

# === Plot ===
plt.hist(data[::-1],
         label=labels[::-1],
         density=Density,
         stacked=Stacked,
         bins=Bins,
         range=Range,
         histtype='step',
         linewidth=2)

# Titles
plt.title(r'$D_s^{+} \rightarrow [D^{0} \rightarrow K^{-} \pi^{+}] e^{+} \nu_{e}$' + '\n' + r'$\Delta m_{\pi}(D_s^{+} - D^{0}) \notin [0.142,\; 0.15] \; \mathrm{GeV}/c^{2}$', loc="left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc="right")

# Labels
plt.xlabel(r'$\Delta m_{e}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
plt.ylabel(r'$Entries/(\;{:.2f}\;MeV/c^2)$'.format(perBin))
plt.legend()
plt.show()

In [ ]:
# Stacked = False
# Density = False
# Bins = 50
# i = 'Ds_diff_D0pi'
# # i = 'Ds_massDifference_0'
# # i = 'Ds_Dstar0Mode1_M_Correction'
# Range = [0.1, 0.55]
# dM = 0.1
# Hits = -1
# perBin = ((Range[1] - Range[0])/Bins)*1000
# print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# labels=[
#         r"$genMotherPDG(D^{0})=423$",
#         r"$genMotherPDG(D^{0})=413$",
#         r"$genMotherPDG(D^{0})=0$",
#         r"$genMotherPDG(D^{0})=23$",
#         r"Other"
#         ]
# colors = ["#1f77b4",  # strong blue
#           "#d62728",  # strong red
#           "#2ca02c",  # strong green
#           "#6a3d9a",  # dark purple
#           "#2E2E2E"]  # charcoal black
# data=[
#       DataFrames["All"][(DataFrames["All"]["Ds_mcPDG"].isna()) & (abs(DataFrames["All"]["D0_genMotherPDG"])==423)  & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM)][i],
#       DataFrames["All"][(DataFrames["All"]["Ds_mcPDG"].isna()) & (abs(DataFrames["All"]["D0_genMotherPDG"])==413)  & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM)][i],
#       DataFrames["All"][(DataFrames["All"]["Ds_mcPDG"].isna()) & (abs(DataFrames["All"]["D0_genMotherPDG"])==0)    & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM)][i],
#       DataFrames["All"][(DataFrames["All"]["Ds_mcPDG"].isna()) & (abs(DataFrames["All"]["D0_genMotherPDG"])==23)   & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM)][i],
#       DataFrames["All"][(DataFrames["All"]["Ds_mcPDG"].isna()) & ((abs(DataFrames["All"]["D0_genMotherPDG"])!=423) & (abs(DataFrames["All"]["D0_genMotherPDG"])!=413) & (abs(DataFrames["All"]["D0_genMotherPDG"])==0) & (abs(DataFrames["All"]["D0_genMotherPDG"])==23)) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM)][i],
#       ]
# # (DataFrames["All"]["D0_isSignal"]==0) & 

# # factor = 0.15
# # plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['e_nPXDHits']>Hits)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['e_nPXDHits']>Hits)][i]), ls='--', linewidth=1.5)
# plt.hist(data, label=labels, color=colors, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# # plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# # plt.axvline(0.15,ls='--',color='gray')

# # Title
# #--------
# plt.title(r'$NaN$', loc = "left")
# # plt.title(r'$Comb. Ds$', loc = "right")
# # Label
# #-------
# plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# plt.xlabel(r'$m(D_{sig}^{0} \gamma_{ROE})\;[GeV/c^{2}]$')
# # plt.yscale("log")
# # plt.xscale("log")
# # plt.ylim(0, 20000)
# plt.legend(loc='upper left', bbox_to_anchor=(1.0, 0.5), frameon=False)
# # plt.tight_layout()
# plt.show()